# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
M. Samland  ->  M. Samland  |  ['M. Samland']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Liu  ->  J. Liu  |  ['J. Liu']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Arxiv has 51 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.13216


extracting tarball to tmp_2502.13216...

 done.


Found 72 bibliographic references in tmp_2502.13216/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2502.13610


...

Found 90 bibliographic references in tmp_2502.13610/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2502.13644


extracting tarball to tmp_2502.13644...

 done.
Retrieving document from  https://arxiv.org/e-print/2502.13866


extracting tarball to tmp_2502.13866... done.


Issues with the citations
list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.13216-b31b1b.svg)](https://arxiv.org/abs/2502.13216) | **Robust machine learning model of inferring the ex-situ stellar fraction of galaxies from photometric data**  |
|| R. Cai, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-02-20*|
|*Comments*| *Accepted in A&A*|
|**Abstract**|            We search for parameters defined from photometric images to quantify the ex situ stellar mass fraction of galaxies. We created mock images using galaxies in the cosmological hydrodynamical simulations TNG100, EAGLE, and TNG50 at redshift $z=0$. We define a series of parameters describing their structures. In particular, the inner and outer halo of a galaxy are defined by sectors ranging from $45-135$ degrees from the disk major axis, and with radii ranging from $3.5-10$ kpc and $10-30$ kpc, respectively, to avoid the contamination of disk and bulge. The surface brightness and colour gradients are defined by the same sectors along the minor axis and with similar radii ranges. We used the Random Forest method to create a model that predicts $f_{\rm exsitu}$ from morphological parameters. The model predicts $f_{\rm exsitu}$ well with a scatter smaller than 0.1 compared to the ground truth in all mass ranges. The models trained from TNG100 and EAGLE work similarly well and are cross-validated; they also work well in making predictions for TNG50 galaxies. The analysis using Random Forest reveals that $\nabla \rho_{\rm outer}$, $\nabla (g-r)_{\rm outer}$, $f_{\rm outerhalo}$ and $f_{\rm innerhalo}$ are the most influential parameters in predicting $f_{\rm exsitu}$, underscoring their significance in uncovering the merging history of galaxies. We further analyse how the quality of images will affect the results by using SDSS-like and HSC-like mock images for galaxies at different distances. Our results can be used to infer the ex situ stellar mass fractions for a large sample of galaxies from photometric surveys.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.13610-b31b1b.svg)](https://arxiv.org/abs/2502.13610) | **HCN and C2H2 in the atmosphere of a T8.5+T9 brown dwarf binary**  |
|| E. C. Matthews, et al. -- incl., <mark>P. Mollière</mark>, <mark>M. Samland</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-02-20*|
|*Comments*| *Accepted for publication in The Astrophysical Journal Letters. 13 pages (5 figures) + appendices*|
|**Abstract**|            T-type brown dwarfs present an opportunity to explore atmospheres teeming with molecules such as H2O, CH4 and NH3, which exhibit a wealth of absorption features in the mid-infrared. With JWST, we can finally explore this chemistry in detail, including for the coldest brown dwarfs that were not yet discovered in the Spitzer era. This allows precise derivations of the molecular abundances, which in turn informs our understanding of vertical transport in these atmospheres and can provide clues about the formation of cold brown dwarfs and exoplanets. This study presents the first JWST/MRS mid-IR spectrum (R ~ 1500-3000) of a T-dwarf: the T8.5+T9 brown dwarf binary WISE J045853.90+643451.9. We fit the spectrum using a parameterized P-T profile and free molecular abundances (i.e., a retrieval analysis), treating the binary as unresolved. We find a good fit with a cloud-free atmosphere and identify H2O, CH4 and NH3 features. Moreover, we make the first detections of HCN and C2H2 (at 13.4$\sigma$ and 9.5$\sigma$ respectively) in any brown dwarf atmosphere. The detection of HCN suggests intense vertical mixing ($K_{zz}\sim10^{11}$cm$^2$s$^{-1}$), challenging previous literature derivations of $K_{zz}$ values for T-type brown dwarfs. Even more surprising is the C2H2 detection, which cannot be explained with existing atmospheric models for isolated objects. This result challenges model assumptions about vertical mixing, and/or our understanding of the C2H2 chemical network, or might hint towards a more complex atmospheric processes such as magnetic fields driving aurorae, or lightning driving ionization. These findings open a new frontier in studying carbon chemistry within brown dwarf atmospheres.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.13866-b31b1b.svg)](https://arxiv.org/abs/2502.13866) | **Hierarchical accretion flow from the G351 infrared dark filament to its central cores**  |
|| <mark>H. Beuther</mark>, et al. |
|*Appeared on*| *2025-02-20*|
|*Comments*| *7 pages, 4 figures, accepted for Astronomy & Astrophysics*|
|**Abstract**|            Aims: We characterize and quantify this multi-scale flow for a prototypical high-mass star-forming region. Methods: In a multi-scale analysis from parsec to ~50au scales, we combined multiple single-dish and interferometric observations to study the gas flow from large-scale sizes of several parsec (Mopra) via intermediate-scale filamentary gas flows (ALMA-IMF) to the central cores (ALMA DIHCA and configuration 10 data). The highest-resolution multi-configuration ALMA dataset achieved a spatial resolution of 0.027''x0.022'' or 50au. Results: This multi-scale study allows us to follow the gas from the environment of the high-mass star-forming region (~2pc) via intermediate-scale (~0.25pc) filamentary gas flows down to the innermost cores within the central few 1000au. The intermediate-scale filaments connect spatially and kinematically to the larger-scale cloud as well as the innermost cores. We estimate a filamentary mass inflow rate around 10^-3M_sun/yr, feeding into the central region that hosts at least a dozen mm cores. While the flow from the cloud via the filaments down to 10^4au appears relatively ordered, within the central 10^4au the kinematic structures become much more complicated and disordered. We speculate that this is caused by the interplay of the converging infalling gas with feedback processes from the forming central protostars. Conclusions: This multi-scale study characterises and quantifies the hierarchical gas flow from clouds down to the central protostars for a prototypical infrared dark cloud with several embedded cores at an unprecedented detail. While comparatively ordered gas flows are found over a broad range of scales, the innermost area exhibits more disordered structures, likely caused by the combination of inflow, outflow and cluster dynamical processes.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.13644-b31b1b.svg)](https://arxiv.org/abs/2502.13644) | **Dynamics of $Z_N$ domain walls with bias directions**  |
|| Y.-J. Li, <mark>J. Liu</mark>, Z.-K. Guo |
|*Appeared on*| *2025-02-20*|
|*Comments*| **|
|**Abstract**|            The spontaneous breaking of a discrete symmetry can lead to the formation of domain walls in the early Universe. In this work, we explore the impact of bias directions on the dynamics of $Z_N$ domain walls, mainly focusing on the $N = 3$ model with a biased potential. Utilizing the Press-Ryden-Spergel method, we numerically investigate the dynamics of domain walls with lattice simulations. We find notable differences in the dynamics of domain walls due to bias directions. Our results indicate that the annihilation time depends not only on the vacuum energy difference $\delta V$ but also on bias directions described by the relative potential difference $ \zeta $.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2502.13216/./figure/picversion12/simulation/fig1_1.png', 'tmp_2502.13216/./figure/picversion12/simulation/fig1_2.png', 'tmp_2502.13216/./figure/picversion12/simulation/fig1_3.png', 'tmp_2502.13216/./figure/picversion12/simulation/TNG100fexSub.png', 'tmp_2502.13216/./figure/picversion12/simulation/EAGLEfexSub.png', 'tmp_2502.13216/./figure/picversion12/simulation/TNG100testfexSub.png', 'tmp_2502.13216/./figure/picversion12/simulation/EAGLEtestfexSub.png', 'tmp_2502.13216/./figure/picversion12/TNG100SDSSlikeID6.png', 'tmp_2502.13216/./figure/picversion12/TNG100SDSSlikeID6_1.png']
copying  tmp_2502.13216/./figure/picversion12/simulation/fig1_1.png to _build/html/
copying  tmp_2502.13216/./figure/picversion12/simulation/fig1_2.png to _build/html/
copying  tmp_2502.13216/./figure/picversion12/simulation/fig1_3.png to _build/html/
copying  tmp_2502.13216/./figure/picversion12/simulation/TNG100fexSub.png to _build/html/
copying  tmp_2502.13216/./figure/picversion12/simulat

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\dlens}{D_{\rm L}}$
$\newcommand{\dsource}{D_{\rm S}}$
$\newcommand{\dls}{D_{\rm S}-D_{\rm L}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand{\vt}{{\rm v}_t}$
$\newcommand{\swd}{Schwarzschild }$
$\newcommand{\atlas}{ATLAS^{\rm{3D}}}$
$\newcommand{\sfg}{S^4G }$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\dgr}{^\circ}$
$\newcommand{\kmsM}{km s^{-1} Mpc^{-1}}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Msunpcsq}{M_\odot pc^{-2}}$
$\newcommand{\Msunpccube}{M_\odot pc^{-3}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\Lsunpcsq}{L_\odot pc^{-2}}$
$\newcommand{\Lsunpccube}{L_\odot pc^{-3}}$
$\newcommand{\MLsun}{M_\odot/L_\odot}$
$\newcommand{\magarcsq}{\mathrm{mag arcsec^{-2}}}$
$\newcommand{\Mstar}{\ensuremath{M_*}}$
$\newcommand{\rmaxhot}{\ensuremath{r|_{{\rm max}(p_{\rm hot})}}}$
$\newcommand{\rcut}{\ensuremath{r_\mathrm{cut}}}$
$\newcommand{\rmax}{\ensuremath{r_\mathrm{max}}}$
$\newcommand{\Mshalo}{\ensuremath{M_{*,\mathrm{halo(r< 2R_e)}}}}$
$\newcommand{\Mswarm}{\ensuremath{M_{*,\mathrm{warm(r< 2R_e)}}}}$
$\newcommand{\LZ}[1]{{\color{magenta} #1}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# Robust machine learning model of inferring the ex situ stellar fraction of galaxies from photometric data

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.13216-b31b1b.svg)](https://arxiv.org/abs/2502.13216)<mark>Appeared on: 2025-02-20</mark> -  _Accepted in A&A_

</div>
<div id="authors">

R. Cai, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** We search for parameters defined from photometric images to quantify the ex situ stellar mass fraction of galaxies.   We created mock images using galaxies in the cosmological hydrodynamical simulations TNG100, EAGLE, and TNG50 at redshift $z=0$ . We define a series of parameters describing their structures, including: the absolute magnitude in $r$ and $g$ bands ( $M_r$ , $M_g$ ), the half-light and 90 \% -light radius ( $r_{50}$ , $r_{90}$ ), the concentration ( $C$ ), the luminosity fractions of inner and outer halos ( $f_{\rm innerhalo}$ , $f_{\rm outerhalo}$ ), the inner and outer surface brightness gradients ( $\nabla \rho_{\rm inner}$ , $\nabla \rho_{\rm outer}$ ), and $*g-r*$ colour gradients ( $\nabla \rm (*g-r*)_{inner}$ , $\nabla \rm (*g-r*)_{outer}$ ). In particular, the inner and outer halo of a galaxy are defined by sectors ranging from $45-135$ degrees from the disk major axis, and with radii ranging from $3.5-10$ kpc and $10-30$ kpc, respectively, to avoid the contamination of disk and bulge. The surface brightness and colour gradients are defined by the same sectors along the minor axis and with similar radii ranges. We used the Random Forest method to create a model that predicts $f_{\rm exsitu}$ from morphological parameters. The model predicts $f_{\rm exsitu}$ well with a scatter smaller than 0.1 compared to the ground truth in all mass ranges. The models trained from TNG100 and EAGLE work similarly well and are cross-validated; they alsowork well in making predictions for TNG50 galaxies. The analysis using Random Forest reveals that $\nabla \rho_{\rm outer}$ , $\nabla \rm (*g-r*)_{outer}$ , $f_{\rm outerhalo}$ and $f_{\rm innerhalo}$ are the   most influential parameters in predicting $f_{\rm exsitu}$ , underscoring their significance in uncovering the merging history of galaxies. We further analyse how the quality of images will affect the results by using SDSS-like and HSC-like mock images for galaxies at different distances. Our results can be used to infer the ex situ stellar mass fractions for a large sample of galaxies from photometric surveys.

</div>

<div id="div_fig1">

<img src="tmp_2502.13216/./figure/picversion12/simulation/fig1_1.png" alt="Fig4.1" width="33%"/><img src="tmp_2502.13216/./figure/picversion12/simulation/fig1_2.png" alt="Fig4.2" width="33%"/><img src="tmp_2502.13216/./figure/picversion12/simulation/fig1_3.png" alt="Fig4.3" width="33%"/>

**Figure 4. -** The correlations between the parameters extracted from mock SDSS photometric observations for galaxies at 40 Mpc and the ex situ stellar mass fraction of galaxies. The grey dots are TNG100 galaxies, the black solid and dashed curves are the running median and the $\pm 1 \sigma$ scatter of the TNG100 galaxies. The blue symbols for EAGLE galaxies. {The Spearman’s rank coefficient($\rho$)} of each correlation is labelled in the figure.
 (*fig3:fexsitufeature*)

</div>
<div id="div_fig2">

<img src="tmp_2502.13216/./figure/picversion12/simulation/TNG100fexSub.png" alt="Fig6.1" width="25%"/><img src="tmp_2502.13216/./figure/picversion12/simulation/EAGLEfexSub.png" alt="Fig6.2" width="25%"/><img src="tmp_2502.13216/./figure/picversion12/simulation/TNG100testfexSub.png" alt="Fig6.3" width="25%"/><img src="tmp_2502.13216/./figure/picversion12/simulation/EAGLEtestfexSub.png" alt="Fig6.4" width="25%"/>

**Figure 6. -** The model predicted $f_{\rm exsitu}$ vs. the ground truth. The different panels are models trained and validated by TNG100 and EAGLE in the top, and cross-validated with each other in the bottom, all with the combined parameters Sub0. For each column, we show the one-to-one comparison in the top panel: $r$ is $\sqrt{r^2}$, and $\sigma(\Delta f_{\rm exsitu})$ is the scatter of residual $\Delta f_{\rm exsitu}=f_{\rm exsitu,predicted}-f_{\rm exsitu,truth}$, the solid black line marks $y=x$, and the dashed black line represents ±1$\sigma(\Delta f_{\rm exsitu})$ scatter. The inset panel is the histogram of $\Delta f_{\rm exsitu}$. In the bottom panel, we show the residual $\Delta f_{\rm exsitu}$ as a function of galaxy $f_{\rm exsitu,truth}$: the red solid and dashed curves show the running median and ±1$\sigma(\Delta f_{\rm exsitu})$ scatter.  (*fig:fex_fextrue*)

</div>
<div id="div_fig3">

<img src="tmp_2502.13216/./figure/picversion12/TNG100SDSSlikeID6.png" alt="Fig1.1" width="50%"/><img src="tmp_2502.13216/./figure/picversion12/TNG100SDSSlikeID6_1.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** A SDSS-like $r$-band image created from a TNG100 galaxy subhalo 6 at z=0 projected near edge-on, and placing it at the distance of 40 Mpc. ** Top:** 2D image. The sector enclosed by blue is defined as the inner halo (3.5 kpc-10 kpc) and that in red is the outer halo (10 kpc-30 kpc). ** Bottom:** surface brightness profile along the minor axis. The black horizontal line indicates the background noise of the sky $\Sigma_{r,0}$, the red and magenta vertical lines mark $r_{\rm 50}$ and $r_{\rm 90}$ obtained from the petrosian radius.
 (*TNG100SDSSlikeID6*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.13216"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\topfraction}{.99}$
$\newcommand{\floatpagefraction}{0.99}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\w0458}{WISE-0458}$
$\newcommand{\Mjup}{\ensuremath{M_\textrm{Jup}}}$
$\newcommand{\Rjup}{\ensuremath{R_\textrm{Jup}}}$</div>



<div id="title">

# $\ce{HCN}$ and $\ce{C2H2}$ in the atmosphere of a T8.5+T9 brown dwarf binary

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.13610-b31b1b.svg)](https://arxiv.org/abs/2502.13610)<mark>Appeared on: 2025-02-20</mark> -  _Accepted for publication in The Astrophysical Journal Letters. 13 pages (5 figures) + appendices_

</div>
<div id="authors">

E. C. Matthews, et al. -- incl., <mark>P. Mollière</mark>, <mark>M. Samland</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** T-type brown dwarfs present an opportunity to explore  atmospheres teeming with molecules such as $\ce{H2O}$ , $\ce{CH4}$ and $\ce{NH3}$ , which exhibit a wealth of absorption features in the mid-infrared. With JWST, we can finally explore this chemistry in detail, including for the coldest brown dwarfs that were not yet discovered in the Spitzer era. This allows precise derivations of the molecular abundances, which in turn informs our understanding of vertical transport in these atmospheres and can provide clues about the formation of cold brown dwarfs and exoplanets. This study presents the first JWST/MRS mid-IR spectrum (R $\sim$ 1500-3000) of a T-dwarf: the T8.5+T9 brown dwarf binary WISE J045853.90+643451.9. We fit the spectrum using a parameterized $P$ - $T$ profile and free molecular abundances (i.e., a retrieval analysis), treating the binary as unresolved. We find a good fit with a cloud-free atmosphere and identify $\ce{H2O}$ , $\ce{CH4}$ and $\ce{NH3}$ features. Moreover, we make the first detections of $\ce{HCN}$ and $\ce{C2H2}$ (at 13.4 $\sigma$ and 9.5 $\sigma$ respectively) in any brown dwarf atmosphere. The detection of $\ce{HCN}$ suggests intense vertical mixing ( $K_{zz}\sim10^{11}$ cm $^2$ s $^{-1}$ ), challenging previous literature derivations of $K_{zz}$ values for T-type brown dwarfs. Even more surprising is the $\ce{C2H2}$ detection, which cannot be explained with existing atmospheric models for isolated objects. This result challenges model assumptions about vertical mixing, and/or our understanding of the $\ce{C2H2}$ chemical network, or might hint towards a more complex atmospheric processes such as magnetic fields driving aurorae, or lightning driving ionization. These findings open a new frontier in studying carbon chemistry within brown dwarf atmospheres.

</div>

<div id="div_fig1">

<img src="tmp_2502.13610/./main_spectrum_plot.png" alt="Fig3.1" width="25%"/><img src="tmp_2502.13610/./main_spectrum_with_model_ch1.png" alt="Fig3.2" width="25%"/><img src="tmp_2502.13610/./main_spectrum_with_model_ch2.png" alt="Fig3.3" width="25%"/><img src="tmp_2502.13610/./main_spectrum_with_model_ch3.png" alt="Fig3.4" width="25%"/>

**Figure 3. -** The MIRI/MRS spectrum of $\w$0458, and our best-fit model. Panel (a) shows the full spectrum, binned to 1/10th the original spectral resolution for visual clarity. Colors represent each individual subchannel of the MRS detector (labelled 1A-3C), and we also label the locations of prominent molecular absorption features in this brown dwarf atmosphere. Panels (b), (d), (f) show the best-fit model (black) compared to the data for each channel (at R=1,000). Our model is a free retrieval using \texttt{pRT}(see Section \ref{sec:retrievals}), including a parameterized pressure-temperature profile and 7 molecular species; 5 molecules are confidently detected (\ce{H2O}, \ce{CH4}, \ce{NH3}, \ce{HCN} and \ce{C2H2}). Panels (c), (e) and (g) show the residuals, taking the 10$^b$ factor into account. (*fig:mainspectrum_and_model*)

</div>
<div id="div_fig2">

<img src="tmp_2502.13610/./opacity_allmolecules.png" alt="Fig5" width="100%"/>

**Figure 5. -** Opacities of molecules in our best-fit model, with abundances matching the best-fit value (for \ce{H2O}, \ce{CH4}, \ce{NH3}, \ce{HCN} and \ce{C2H2}) and 2$\sigma$ upper limits (for CO and $CO_2$) from our best-fit model. Opacities in the upper panel are at R=100 and in the lower panel are at R=1000 (matching the resolution used in our \texttt{pRT} retrievals). In the lower panel we highlight the regions where \ce{HCN} and \ce{C2H2} absorb most strongly in the near- and mid-IR. For the $\w$0458 abundances, both species are only detectable in the mid-IR around 14 \micron in the $\w$0458 atmosphere. With sufficiently high abundances (higher than those observed here), both species might become detectable in the near-IR, at 3 \micron and 1.5 \micron. (*fig:hcnopacity*)

</div>
<div id="div_fig3">

<img src="tmp_2502.13610/./HCN_C2H2_modelcomparison_v3.png" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison of the data and best-fit models with and without including the molecules \ce{HCN} and \ce{C2H2}. Here we show our retrieved spectra for a "basic" model with only \ce{H2O}, \ce{CH4}, \ce{NH3}, CO and $CO_2$(brown), and a model that also includes \ce{HCN} and \ce{C2H2}(green), alongside the data (black, including our retrieved uncertainty inflation). The lower panel shows the residuals (data - model) for the both models, and shading in the background of both panels indicates regions of high opacity for \ce{HCN}(pink) and \ce{C2H2}(blue). The plot shows only the 13.4-14.6 \micron region of the spectrum where \ce{HCN} and \ce{C2H2} absorption features are significant (see also Fig. \ref{fig:hcnopacity}), but these species have a small continuum effect across more of the mid-IR. (*fig:hcn_c2h2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.13610"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Hierarchical accretion flow from the G351 infrared dark filament to its central cores

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.13866-b31b1b.svg)](https://arxiv.org/abs/2502.13866)<mark>Appeared on: 2025-02-20</mark> -  _7 pages, 4 figures, accepted for Astronomy & Astrophysics_

</div>
<div id="authors">

B. H., et al.

</div>
<div id="abstract">

**Abstract:** Quantifying the accretion flow from large cloud scales down to individual protostars is a central ingredient to the understanding of (high-mass) star formation. We characterize and quantify this multi-scale flow for a prototypical high-mass star-forming region. In a multi-scale analysis from parsec to $\sim$ 50 au scales, we combined multiple single-dish and interferometric observations to study the gas flow from large-scale sizes of several parsec (Mopra) via intermediate-scale filamentary gas flows (ALMA-IMF) to the central cores (ALMA DIHCA and configuration 10 data). The highest-resolution multi-configuration ALMA dataset achieved a spatial resolution of $0.027"\times 0.022"$ or 50 au. This multi-scale study allows us to follow the gas from the  environment of the high-mass star-forming region ( $\sim$ 2 pc) via intermediate-scale ( $\sim$ 0.25 pc) filamentary gas flows down to the innermost cores within the central few 1000 au. The intermediate-scale filaments connect spatially and kinematically to the larger-scale cloud as well as the innermost cores. We estimate a filamentary mass inflow rate around $10^{-3}$ M $_{\odot}$ yr $^{-1}$ , feeding into the central region that hosts at least a dozen mm cores. While the flow from the cloud via the filaments down to 10 $^4$ au appears relatively ordered, within the central 10 $^4$ au the kinematic structures become much more complicated and disordered. We speculate that this is caused by the interplay of the converging infalling gas with feedback processes from the forming central protostars. This multi-scale study characterises and quantifies the hierarchical gas flow from clouds down to the central protostars for a prototypical infrared dark cloud with several embedded cores at an unprecedented detail. While comparatively ordered gas flows are found over a broad range of scales, the innermost area exhibits more disordered structures, likely caused by the combination of inflow, outflow and cluster dynamical processes.

</div>

<div id="div_fig1">

<img src="tmp_2502.13866/./almaimf_zoom.png" alt="Fig3" width="100%"/>

**Figure 3. -** Zoom-in plots from the more central region in G351. The two left panels show the ALMA-IMF data \citep{motte2022,ginsburg2022,cunningham2023} from the first zoom-region (black box in Fig. \ref{g351_large}). The color-scale in the left and middle panels are the 1st moment maps in $H_2$CO and DCN. The contours show the 1.3 mm continuum emission starting at 4$\sigma$ and continue in 8$\sigma$ steps (1$\sigma$=0.6 mJy beam$^{-1}$). The ellipses in the middle panel outline the areas used for the mass flow rate estimates, and the red arrow shows the direction of the red-shifted outflow lobe from \citet{beuther2017b}. The right panel then shows the final zoom-in (magenta boxes in middle panel and Fig. \ref{g351_large}) data from the DIHCA and high-resolution studies \citep{olguin2021,beuther2019} with the $H_2$CO 1st moment map in color-scale and the 1.3 mm continuum data in contours (5 to 25$\sigma$ in 5$\sigma$ steps with 1$\sigma$=60 $\mu$Jy beam$^{-1}$). The arrow in the right panel marks the direction of the position-velocity cut in Fig. \ref{pv_h2co}. Linear scale-bars are shown in the middle and right panels, synthesized beams are presented in the bottom- and top-left corners of the respective panels. (*ALMA-IMF*)

</div>
<div id="div_fig2">

<img src="tmp_2502.13866/./zoom_h2co.png" alt="Fig4" width="100%"/>

**Figure 4. -** Velocity structure of the central region. The left three panels show integrated $H_2$CO$(3_{2,2}-2_{2,1})$ maps for different velocity regimes as marked. The right panel presents a corresponding position-velocity cut from east to west along the line shown in the right panel of Fig. \ref{ALMA-IMF} and in the 2nd panel from left here. The dashed vertical lines in all panels mark the R.A. positions of the two central cores mm1 and mm2 (right and left dashed lines, respectively), where mm1 also drives a molecular outflow in northwest-southeast direction \citep{beuther2019}. The horizontal dashed line in the pv-diagram shows the $v_{\rm LSR}\sim -3.6$ km s$^{-1}$ from \citet{leurini2011}. Other features are marked and labeled.  (*pv_h2co*)

</div>
<div id="div_fig3">

<img src="tmp_2502.13866/./g351_large_v2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Overview of the G351 region. The left panel shows in color-scale the Spitzer 8 $\mu$m emission \citep{churchwell2009} and in contours the ATLASGAL 870 $\mu$m emission \citep{schuller2009}. Contour levels start at the 4$\sigma$ level of 240 mJy beam$^{-1}$. The middle and right panels present the molecular emission from the HCO$^+$(1--0)  and $N_2$H$^+$(1--0) lines observed with the MALT90 survey \citep{jackson2013}. In both cases, the color-scale shows the 1st moment maps (intensity-weighted peak velocities), and the contours present the integrated line emission or 0th moment from 5 to 95\% of the respective peak emission. For $N_2$H$^+$(1--0), the 1st moment map is from the isolated hyperfine component shifted by +8.0 km s$^{-1}$ to the $v_{\rm{lsr}}$. Since the rms increases toward the map edges, we masked the emission outside the 5\% 0th moment map for the $N_2$H$^+$(1--0) map.  The black and magenta boxes in the middle and right panels outline the areas shown in the following ALMA-IMF and ALMA central core images. Linear scale-bars are shown in the left and right panels, and the MALT90 beams are presented in the bottom corners. (*g351_large*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.13866"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

339  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
